# Relu

In [16]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
tf.enable_eager_execution()
import os

In [17]:
import tensorflow as tf
print(tf.__version__)

1.14.0


In [18]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis = -1) #add channel
    test_data = np.expand_dims(test_data, axis = -1)
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)
    return train_data, train_labels, test_data, test_labels
    
def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32)/255.0
    test_data = test_data.astype(np.float32)/255.0
    return train_data, test_data
    

In [31]:
def loss_fn(model, images, labels):
    logits = model(images, training = True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels = labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training = False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

In [20]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init):
    return tf.keras.layers.Dense(units = label_dim, use_bias=True, kernel_initializer=weight_init)

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

## create model

In [21]:
class create_model(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()
        
        self.model = tf.keras.Sequential()
        self.model.add(flatten()) #why? fully connected layer사용할거라
        
        for i in range(2):
            self.model.add(dense(256, weight_init)) #[N,256]
            self.model.add(relu())
            
        self.model.add(dense(label_dim, weight_init)) #[N,10]
        
    def call(self, x, training = None, mask = None):
        x = self.model(x)
        
        return x
        

In [22]:
train_x, train_y, test_x, test_y = load_mnist()

learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x)//batch_size

label_dim = 10

train_flag = True

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
                    shuffle(buffer_size=100000).\
                    prefetch(buffer_size=batch_size).\
                    batch(batch_size).\
                    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
                    shuffle(buffer_size=100000).\
                    prefetch(buffer_size=len(test_x)).\
                    batch(len(test_x)).\
                    repeat()

train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

In [52]:
network = create_model(label_dim)

In [53]:
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

In [54]:
# prevent networks from losing connects or trained info
checkpoint = tf.train.Checkpoint(dnn=network)

for epoch in range(training_epochs):
    for idx in range(training_iterations):
        train_input, train_label = train_iterator.get_next()
        
        with tf.GradientTape() as tape:
            predictions = network(train_input)
#            loss = loss_object(predictions, train_label)     # 위의 설명 2. 를 참고하여 None을 채우세요.
            loss = loss_fn(network, train_input, train_label)
        grads = tape.gradient(loss, network.variables)
        optimizer.apply_gradients(grads_and_vars = zip(grads, network.variables))
        
        train_loss = loss_fn(network, train_input, train_label)
        train_accuracy = accuracy_fn(network, train_input, train_label)
        
        test_input, test_label = test_iterator.get_next()
        test_accuracy = accuracy_fn(network, test_input, test_label)
        
        print("Epoch: [%2d] [%5d/%5d], train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f"\
             % (epoch, idx, training_iterations, train_loss, train_accuracy, test_accuracy))


Epoch: [ 0] [    0/  468], train_loss: 2.17042923, train_accuracy: 0.2656, test_Accuracy: 0.1997
Epoch: [ 0] [    1/  468], train_loss: 2.08910751, train_accuracy: 0.4297, test_Accuracy: 0.2865
Epoch: [ 0] [    2/  468], train_loss: 2.01778126, train_accuracy: 0.3750, test_Accuracy: 0.3583
Epoch: [ 0] [    3/  468], train_loss: 1.96049321, train_accuracy: 0.5078, test_Accuracy: 0.4242
Epoch: [ 0] [    4/  468], train_loss: 1.84629560, train_accuracy: 0.6172, test_Accuracy: 0.4799
Epoch: [ 0] [    5/  468], train_loss: 1.84598053, train_accuracy: 0.5547, test_Accuracy: 0.5492
Epoch: [ 0] [    6/  468], train_loss: 1.74395049, train_accuracy: 0.6328, test_Accuracy: 0.6004
Epoch: [ 0] [    7/  468], train_loss: 1.62881565, train_accuracy: 0.6797, test_Accuracy: 0.6385
Epoch: [ 0] [    8/  468], train_loss: 1.51331866, train_accuracy: 0.7266, test_Accuracy: 0.6716
Epoch: [ 0] [    9/  468], train_loss: 1.43385494, train_accuracy: 0.7109, test_Accuracy: 0.7002
Epoch: [ 0] [   10/  468], tra

Epoch: [ 0] [   87/  468], train_loss: 0.41007906, train_accuracy: 0.8906, test_Accuracy: 0.9053
Epoch: [ 0] [   88/  468], train_loss: 0.34035683, train_accuracy: 0.8984, test_Accuracy: 0.9020
Epoch: [ 0] [   89/  468], train_loss: 0.45941332, train_accuracy: 0.8516, test_Accuracy: 0.9063
Epoch: [ 0] [   90/  468], train_loss: 0.32666248, train_accuracy: 0.8984, test_Accuracy: 0.9075
Epoch: [ 0] [   91/  468], train_loss: 0.30755290, train_accuracy: 0.9297, test_Accuracy: 0.9036
Epoch: [ 0] [   92/  468], train_loss: 0.36861593, train_accuracy: 0.8906, test_Accuracy: 0.9028
Epoch: [ 0] [   93/  468], train_loss: 0.50677133, train_accuracy: 0.8594, test_Accuracy: 0.9050
Epoch: [ 0] [   94/  468], train_loss: 0.51624763, train_accuracy: 0.8281, test_Accuracy: 0.9117
Epoch: [ 0] [   95/  468], train_loss: 0.26408342, train_accuracy: 0.9297, test_Accuracy: 0.9143
Epoch: [ 0] [   96/  468], train_loss: 0.18952349, train_accuracy: 0.9453, test_Accuracy: 0.9118
Epoch: [ 0] [   97/  468], tra

KeyboardInterrupt: 

In [ ]:
print("test_Accuracy: %.4f" % (test_accuracy))

In [ ]:
checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))